In [156]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
captions_path = "./Flickr8k.token.txt"
caption_file = open( captions_path, 'r' )
captions = caption_file.read()

In [3]:
dataset = {}

for line in captions.split('\n'):
    tokens = line.split('\t')
    image_id = tokens[0].split('.')[0]
    image_caption = tokens[-1]
    if image_id not in dataset:
        dataset[image_id] = []
    dataset[image_id].append(image_caption)

In [4]:
import string

translator = str.maketrans('', '', string.punctuation)

for _, captions_ in dataset.items():
    for i in range(len(captions_)):
        caption_ = captions_[i]
        caption_ = caption_.translate(translator)
        caption_ = caption_.lower()
        captions_[i] = caption_

In [5]:
dataset

{'': [''],
 '492802403_ba5246cfea': ['one dog jumps up to catch a ball as another dog runs towards it ',
  'two dogs are playing in a fenced in yard ',
  'two dogs play catch with a rubber ball ',
  'two large dogs are playing together in a yard with a tall fence ',
  'two tan dogs playing together in a fenced yard '],
 '494792770_2c5f767ac0': ['a brown dog walks in the under brush ',
  'a dog pauses in the bushes ',
  'a dog runs through the long grass ',
  'brown dog with mouth open standing amidst greenery ',
  'brown longhaired dog in garden '],
 '2735158990_56ff6bf9b0': ['a brown and white dog stands on his hind leg and barks ',
  'a brown dog on his hind legs in the shadow of some trees ',
  'a dog jumps in the woods ',
  'a dog stands on its hind legs in a wooded area while another dog watches ',
  'a white dog is jumping in the air to catch a ball in the woods '],
 '2730938963_c4ed3e2258': ['three boys leaping into a swimming pool ',
  'three happy boys wearing blue bathing sui

In [6]:
vocab = set()
for _, captions_ in dataset.items():
    for caption_ in captions_:
        vocab.update(caption_.split())

In [7]:
print(len(vocab))

8828


In [8]:
all_captions = []
for _, captions_ in dataset.items():
    for caption_ in captions_:
        all_captions.append(caption_)

In [9]:
#we will take in account word frequency of 10 words

freq = 10
word_dict = {}

for caption_ in all_captions:
    for word in caption_.split():
        word_dict[word] = word_dict.get(word, 0) + 1

In [10]:
vocab = [word for word in word_dict if word_dict[word] >= freq]

In [11]:
print(len(vocab))

1955


In [12]:
#1955 unique words with frequency >= 10 and without a padding space

In [13]:
trainset_path = './Flickr_8k.trainImages.txt'
trainset_file = open(trainset_path, 'r')
trainset_ = trainset_file.read()
trainset = []
for image_id in trainset_.split('\n'):
    image_ = image_id.split('.')[0]
    trainset.append(image_)
print('trainset: %d' % len(trainset))

trainset: 6001


In [14]:
train_dataset = {}

for image_id in trainset:
    train_dataset[image_id] = ['startseq ' + caption_ + ' endseq' for caption_ in dataset[image_id]]

In [15]:
train_dataset

{'': ['startseq  endseq'],
 '492802403_ba5246cfea': ['startseq one dog jumps up to catch a ball as another dog runs towards it  endseq',
  'startseq two dogs are playing in a fenced in yard  endseq',
  'startseq two dogs play catch with a rubber ball  endseq',
  'startseq two large dogs are playing together in a yard with a tall fence  endseq',
  'startseq two tan dogs playing together in a fenced yard  endseq'],
 '2735158990_56ff6bf9b0': ['startseq a brown and white dog stands on his hind leg and barks  endseq',
  'startseq a brown dog on his hind legs in the shadow of some trees  endseq',
  'startseq a dog jumps in the woods  endseq',
  'startseq a dog stands on its hind legs in a wooded area while another dog watches  endseq',
  'startseq a white dog is jumping in the air to catch a ball in the woods  endseq'],
 '3359587274_4a2b140b84': ['startseq a man skis while wearing a parachute  endseq',
  'startseq a parachute skier lifting off from the ground  endseq',
  'startseq a person a

In [16]:
len(train_dataset)

6001

In [125]:
model = InceptionV3(weights='imagenet')
model_ = Model(model.input, model.layers[-2].output)

In [127]:
def preprocess(image_path):
    
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [130]:
def encode(image_path):
    image = preprocess(image_path) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [152]:
for id,_ in train_dataset.items():
    if id is '':
        print(p)

In [159]:
start = time()
encoding_train = {}
for id,_ in train_dataset.items():
    path = './Flicker8k_Dataset/' + id + '.jpg'
    encoding_train[id] = encode(path)
    print("Time taken in seconds =", time()-start)

Time taken in seconds = 0.5170130729675293
Time taken in seconds = 1.0236835479736328
Time taken in seconds = 1.572420358657837
Time taken in seconds = 2.12967848777771
Time taken in seconds = 2.694650411605835
Time taken in seconds = 3.2315986156463623
Time taken in seconds = 3.7267913818359375
Time taken in seconds = 4.232088327407837
Time taken in seconds = 4.7568769454956055
Time taken in seconds = 5.259743690490723
Time taken in seconds = 5.760981559753418
Time taken in seconds = 6.340948820114136
Time taken in seconds = 7.119978427886963
Time taken in seconds = 7.72334098815918
Time taken in seconds = 8.260797500610352
Time taken in seconds = 8.80447793006897
Time taken in seconds = 9.336833477020264
Time taken in seconds = 10.041603088378906
Time taken in seconds = 10.834113121032715
Time taken in seconds = 11.446977853775024
Time taken in seconds = 12.03317379951477
Time taken in seconds = 12.552376508712769
Time taken in seconds = 13.134905815124512
Time taken in seconds = 13.

KeyboardInterrupt: 

In [20]:
len(vocab)

1955

In [21]:
index_to_words = {}
words_to_index = {}

index = 1
for word in vocab:
    words_to_index[word]  = index
    index_to_words[index] = word
    index+=1
    

In [22]:
#max length of a caption
max_length = 0
for caption_ in all_captions:
    length_of_caption = len(caption_.split())
    if max_length <= length_of_caption:
        max_length = length_of_caption

In [23]:
max_length

36

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def generator_(dataset, images, words_to_index, max_length, batch_size):
    
    n=0
    images_list = []
    input_cap_list = []
    output_cap_list = []
    while True:
        for key, caption_ in dataset.items():
            n+=1
            
            image = images[key]
            for cap in caption_:
                for word in cap.split():
                    if word in words_to_index:
                        coded = [words_to_index[word]]
                for i in range(1, len(coded)):
                    input_, output_ = seq[:i], seq[i]
                    input_ = pad_sequences([input_], maxlen = max_length)[0]
                    output_ = to_categorical([output_], num_classes = vocab)[0]
                    # store
                    images_list.append(image)
                    input_cap_list.append(input_)
                    output_cap_list.append(output_)
            if n==batch_size:
                yield [[array(images_list), array(input_cap_list)], array(output_cap_list)]
                images_list = []
                input_cap_list = []
                output_cap_list = []
                n=0

In [ ]:
embeddings = {}
file = open('./glove.6B.50d.txt', encoding="utf-8")

for item in file:
    single_embed = item.split()
    word = single_embed[0]
    embedding = np.asarray(single_embed[1:], dtype='float32')
    embeddings[word] = embedding
file.close()

In [104]:
embeddings

{'dictator': array([ 0.46369 , -0.67047 ,  0.53768 , -0.21587 ,  0.013963, -1.2744  ,
         1.1698  , -0.024596, -1.2068  ,  0.98465 ,  0.050475,  0.50897 ,
        -0.94696 ,  0.12911 ,  0.99687 , -0.34007 , -0.73889 ,  0.1079  ,
         0.75472 ,  1.1269  , -1.2703  ,  0.32013 , -0.18354 , -0.98221 ,
        -0.774   , -2.0994  , -0.42576 , -0.70225 ,  0.2554  ,  1.4157  ,
         1.0021  , -0.67083 , -1.4793  ,  0.017117, -0.58276 ,  0.80443 ,
        -0.3068  ,  0.17488 , -0.38247 , -0.24949 , -0.45986 ,  1.3966  ,
         0.18562 , -0.15566 , -0.091126,  0.28989 , -1.1697  , -1.8718  ,
        -0.47837 , -1.0175  ], dtype=float32),
 'rignall': array([-0.19231  , -0.89983  , -0.16324  , -0.26004  , -0.0085374,
        -0.52014  ,  0.33074  , -0.21157  , -0.43959  ,  0.6976   ,
         0.25979  ,  0.37416  , -0.43903  , -0.0014049,  0.12807  ,
        -0.27914  , -0.75635  , -0.43228  ,  0.42757  ,  0.77579  ,
        -0.39056  ,  0.088645 , -0.26764  ,  0.35711  ,  0.80596  

In [110]:
embed_dim = 50
word_vec_glove = np.zeros((len(vocab)+1, embed_dim))

for word, i in words_to_index.items():
    embed_vec = embeddings.get(word)
    if embed_vec is not None:
        word_vec_glove[i] = embed_vec

In [ ]:
image_input = Input(shape=(2048,))
fc1 = Dropout(0.5)(image_input)
fc2 = Dense(256, activation='relu')(fc1)
caption_input = Input(shape=(max_length,))
seq_1 = Embedding(len(vocab)+1, embed_dim, mask_zero=True)(caption_input)
seq_2 = Dropout(0.5)(seq_1)
seq_3 = LSTM(256)(seq_2)
decoder_1 = add([fc2, seq_3])
decoder_2 = Dense(256, activation='relu')(decoder_1)
outputs = Dense(len(vocab)+1, activation='softmax')(decoder_2)
model1 = Model(inputs=[image_input, caption_input], outputs=outputs)

In [121]:
model1.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 36, 50)       97800       input_8[0][0]                    
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 2048)         0           input_7[0][0]                    
__________________________________________________________________________________________________
dropout_7 

In [ ]:
print(model.layers[2])
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.optimizer.lr = 0.0001
epochs = 10
batch_size = 6
steps = len(train_dataset)//batch_size
for i in range(epochs):
    generator = generator_(train_dataset, image_vector, words_to_index, max_length, batch_size)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')